# Projeto 1 - Ciência dos Dados

Nome: Bruno Conte Paiva

Nome: André Faia Vasconcelos Ramos

___
Carregando algumas bibliotecas:

In [88]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
from IPython.display import display
from nltk.tokenize import TweetTokenizer
import nltk
pd.options.display.max_rows = 900

Carregando o arquivo excel:

In [87]:
import os

filename = 'Projeto_excel.xlsx'

if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
    
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo Projeto_excel.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes, neutros e não relevantes do treinamento:

In [86]:
data_treinamento = pd.read_excel(filename, coverters = {'Treino': str})

Dando nomes aos valores de 'relevância' no treinamento:

In [85]:
data_treinamento['relevância'] = data_treinamento['relevância'].astype('category')
data_treinamento.relevância.cat.categories = ['irrelevante', 'neutro', 'relevante']
data_treinamento.relevância.cat = pd.Categorical(data_treinamento.relevância, categories = ['irrelevante', 'neutro', 'relevante'])

Filtrando e organizando os tweets do treinamento por 'relevância':

In [84]:
#para relevantes:

data_treinamento_relevantes = data_treinamento['relevância'] == 'relevante'
treinamento_relevantes = data_treinamento.loc[data_treinamento_relevantes, :]


#para neutros:

data_treinamento_neutros = data_treinamento['relevância'] == 'neutro'
treinamento_neutros = data_treinamento.loc[data_treinamento_neutros, :]

#para irrelevantes:

data_treinamento_irrelevantes = data_treinamento['relevância'] == 'irrelevante'
treinamento_irrelevantes = data_treinamento.loc[data_treinamento_irrelevantes, :]

#todas as relevâncias:

lista_treinamento = [treinamento_irrelevantes, treinamento_neutros, treinamento_relevantes]

Carregando a base de dados com os tweets classificados como relevantes, neutros e não relevantes do teste:

In [83]:
data_teste = pd.read_excel(filename, sheet_name = 'Teste', coverters = {'Test': str})

Dando nomes aos valores de 'relevância' no teste:

In [82]:
data_teste['relevância'] = data_teste['relevância'].astype('category')
data_teste.relevância.cat.categories = ['irrelevante', 'neutro', 'relevante']
data_teste.relevância.cat = pd.Categorical(data_teste.relevância, categories = ['irrelevante', 'neutro', 'relevante'])

Filtrando e organizando os tweets do teste por 'relevância':

In [81]:
#para relevantes:

data_teste_relevantes = data_teste['relevância'] == 'relevante'
teste_relevantes = data_teste.loc[data_teste_relevantes, :]

#para neutros:

data_teste_neutros = data_teste['relevância'] == 'neutro'
teste_neutros = data_teste.loc[data_teste_neutros, :]

#para irrelevantes:

data_teste_irrelevantes = data_teste['relevância'] == 'irrelevante'
teste_irrelevantes = data_teste.loc[data_teste_irrelevantes, :]

lista_teste = [data_teste_irrelevantes, data_teste_neutros, data_teste_relevantes]

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O produto escolhido foi o iPhone que é um smartphone da marca Apple, de relevância mundial. A classificação foi feita a partir da presença ou não de comentários que falam sobre o produto, sem relevar a marca.

Critérios de classificação dos tweets:

*"irrelevantes" : Tweets que não falam sobre o produto e suas funcionalidades, mas citam a marca.

*"neutros" : Tweets que falam sobre a marca, porém nada relevante.

*"relevantes": Tweets que agregam algo à marca, como sugestões, críticas, entre outros.



___
### Criando as funções de clean-up:

In [79]:
#função que realizará o primeiro cleanup, trocando sinais de pontuação por espaços:
def cleanup(textos):
    pontuação = '[!-.:?;@]'
    pattern = re.compile(pontuação)
    text_subbed = re.sub(pattern,'', textos)

    return text_subbed

#função que realizará a remoção dos arrobas('@'):

def arroba(lista_arroba):
    arroba = '@'
    for l in lista_arroba:
        if arroba in l:
            lista_arroba.replace(arroba, "")
    
    return lista_arroba

#função que realizará a remoção de strings de links:

def links(lista_links):
    link = 'http'
    for l in lista_links:
        if link in l:
            lista_links.replace(link, "")
    
    return lista_links

Realizando os cleanups:

In [78]:
#removendo sinais de pontuação:

textos_sinais = []
lista_textos_filtrados = []
for df in lista_treinamento:
    lista_sinais = df['Treino'].values.tolist()
    for t in lista_sinais:
        lista_textos_filtrados = cleanup(t)
        textos_sinais.append(lista_textos_filtrados)

#removendo arrobas:

textos_arrobas = []
lista_textos_filtrados1 = []
for t in textos_sinais:
    lista_arrobas = df
    for df in lista_arrobas:
        lista_textos_filtrados = arroba(df)
        textos_arrobas.append(lista_textos_filtrados1)
    
#removendo links:

textos_links = []
lista_textos_filtrados2 = []
for t in textos_arrobas:
    lista_links = df
    for df in lista_links:
        lista_textos_filtrados1 = links(df)
        textos_links.append(lista_textos_filtrados2)

Iniciando os cálculos das probabilidades no treinamento:

In [77]:
#cálculo das probabilidades no treinamento:

lista_relevâncias = data_treinamento['relevância'].value_counts(True, sort = False)

#probabilidade de ser irrelevante:
P_I = lista_relevâncias[0]

#probabilidade de ser neutro:
P_N = lista_relevâncias[1]

#probabilidade de ser relevante:
P_R = lista_relevâncias[2]

#probabilidade de cada grau de relevância:

lista_relevâncias = [P_I, P_N, P_R]

In [76]:
#listas separadas:

palavras_treinamento_I = pd.Series(lista_textos_filtrados2).value_counts()
palavras_treinamento_N = pd.Series(lista_textos_filtrados2).value_counts()
palavras_treinamento_R = pd.Series(lista_textos_filtrados2).value_counts()

dicionario_palavras = [palavras_treinamento_I, palavras_treinamento_N, palavras_treinamento_R]

### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [72]:
#função do classificador Naive Bayes:

lista = []
dicionario = {}
dicionario2= {}
lista_probabilidades = []
    
#definindo as variáveis e colocando seus respectivos valores:
    
a = 1*10**-5
b = 0

def Classificador(texto):
    texto_filtrado = cleanup(texto)
    lista = []
    for d in dicionario_palavras:
        dicionario = {}
        for p in texto_filtrado:
            if p in dicionario:
                dicionario2[p] = dicionario[p]
            else:
                dicionario2[p] = 0
        lista.append(pd.Series(dicionario2))
        
    #listas para cada grau de relevância:

    I = lista[0]
    N = lista [1]
    R = lista [2]

    Relevância = [I, N, R]

    valor_maximo = 0
    
    for c in range(0, len(lista_probabilidades)):
        if lista_probabilidades[valor_maximo] <= lista_probabilidades[c]:
            valor_maximo = c
            
    if valor_maximo == 0:
        nb_relevância = "irrelevante"

    elif valor_maximo == 1:
        nb_relevância = "neutro"

    else:
        nb_relevância = "relevante"
        
    return nb_relevância

Testando o classificador, montando tabelas:

In [71]:
#finalizando classificador Naive Bayes:

naive_bayes = []

for x in data_treinamento["Treino"]:
    naive_bayes.append(Classificador(x))

data_treinamento["RELEVÂNCIA"] = naive_bayes

#Iniciando as comparações:

RELEVÂNCIA_vs_relevância = pd.crosstab(data_treinamento.relevância, data_treinamento.RELEVÂNCIA, normalize = True)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**